In [1]:
import tensorflow as tf
import numpy as np

In [25]:
batched_inputs = pickle.load(open('../word2vec/batched_inputs.npz', 'rb'))
batched_targets = pickle.load(open('../word2vec/batched_targets.npz', 'rb'))

In [27]:
class corpusClean():

    def __init__(self):
        self.num_re = re.compile('[0-9一二三四五六七八九十]+')
        self.char_re = re.compile('[a-zA-Z\-\,\.\。\，]+')

    def clean(self, line):
        sentence = line
        removed_char = re.sub(self.char_re, ' <CHAR> ', sentence)
        removed_num = re.sub(self.num_re, ' <NUMBER> ', removed_char)
        words = removed_num.split()
        saved_words = ['<BEGIN>']
        for word in words:
            if len(word) == 1 or word.find('<') != -1:
                saved_words.append(word)
            else:
                for w in word:
                    saved_words.append(w)
        saved_words.append('<END>')
        return saved_words
    
    
class layer(object):
    def __init__(self):
        raise NotImplementedError
    
    def __str__(self):
        raise NotImplementedError
    
class affine_layer(layer):
    def __init__(self, name, inputs, shape, activation, reuse=False):
        with tf.variable_scope('affine', reuse=reuse):
            self.weights = tf.get_variable(name = name+'_weights', initializer=tf.truncated_normal(shape, stddev=0.05), dtype=tf.float32)
            self.bias = tf.get_variable(name = name+'_bias', initializer=tf.zeros([shape[-1]]))
            self.outputs = activation(tf.add(tf.matmul(inputs, self.weights), self.bias))
    def __str__(self):
        return self.outputs.__str__()
    
class conv_layer(layer):
    def __init__(self, name, inputs, shape, activation, reuse=False):
        with tf.variable_scope('conv', reuse=reuse):
            self.kernel = tf.get_variable(name = name+'_kernel', initializer=tf.truncated_normal(shape, stddev=0.05), dtype=tf.float32)
            self.bias = tf.get_variable(name = name+'_bias', initializer=tf.zeros([shape[-1]]))
            self.outputs = activation(tf.add(tf.nn.conv2d(inputs, self.kernel, padding='VALID', strides=[1,1,1,1]), self.bias))
    def __str__(self):
        return self.outputs.__str__()
    
class pooling_layer(layer):
    def __init__(self, name, inputs, reuse=False):
        with tf.variable_scope('pooling', reuse=reuse):
            self.outputs = tf.nn.max_pool(name = name+'_maxpooling', value=inputs, ksize=[1,2,2,1], padding='VALID', strides=[1,2,2,1])
    def __str__(self):
        return self.outputs.__str__()
        
class reshape_layer(layer):
    def __init__(self, name, inputs, shape, reuse=False):
        with tf.variable_scope('reshape', reuse=reuse):
            self.outputs = tf.reshape(name=name, tensor=inputs, shape=shape)
    def __str__(self):
        return self.outputs.__str__()
        
class deconv_layer(layer):
    def __init__(self, name, inputs, kernel_shape, output_shape, activation, reuse=False):
        with tf.variable_scope('deconv', reuse=reuse):
            self.kernel = tf.get_variable(name = name+'_kernel', initializer=tf.truncated_normal(kernel_shape, stddev=0.05), dtype=tf.float32)
            self.bias = tf.get_variable(name = name+'_bias', initializer=tf.zeros([kernel_shape[-2]]))
            self.outputs = activation(tf.add(tf.nn.conv2d_transpose(inputs, self.kernel, output_shape=output_shape, padding='VALID', strides=[1,2,2,1]), self.bias))
    def __str__(self):
        return self.outputs.__str__()
    
class lstm_layer(layer):
    def __init__(self, name, inputs, n_units, reuse=False):
        with tf.variable_scope('lstm', reuse=reuse):
            self.outputs, self.states = tf.nn.dynamic_rnn(
                cell = tf.contrib.rnn.BasicLSTMCell(n_units),
                inputs = inputs,
                dtype = tf.float32)
    def __str__(self):
        return self.outputs.__str__()

In [26]:
graph = tf.Graph()
with graph.as_default():
    
    input_placeholder = tf.placeholder(tf.float32, [None, None, 200])
    target_placeholder = tf.placeholder(tf.float32, [None, 134])
    step_size_placeholder = tf.placeholder(tf.int32)
    batch_size_placeholder = tf.placeholder(tf.int32)
    
    
    lstm_layer1 = lstm_layer('lstm1', input_placeholder, 200)
    reshape1 = reshape_layer('reshape1', lstm_layer1.outputs, [batch_size_placeholder, step_size_placeholder*200])
    topk1 = tf.nn.top_k(reshape1.outputs, )
    
    
    affine1 = affine_layer('affine1', lstm_layer1.outputs[:,-1,:], [200, 256], tf.nn.relu)
    affine2 = affine_layer('affine2', affine1.outputs, [256, 256], tf.nn.relu)
    affine3 = affine_layer('affine3', affine2.outputs, [256, 134], tf.identity)
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=affine3.outputs, labels=target_placeholder))
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(affine3.outputs, 1), tf.argmax(target_placeholder, 1)), tf.float32))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    for i in range(500):
        
        accs = 0.0
        errs = 0.0
        n = 0
        for key in batched_inputs.keys():
            inputs_batch = batched_inputs[key]
            targets_batch = batched_targets[key]

            feed_dict = {input_placeholder:inputs_batch, target_placeholder:targets_batch}
            _, acc, err = sess.run([optimizer, accuracy, loss], feed_dict=feed_dict)
            accs += acc
            errs += err
            n += len(inputs_batch)
            break
        print('#Training Epoch %d, acc: %f, err: %f, n: %d'%(i, accs/n, errs/n, n))

[[ 1.42182922  0.16067666  0.0759489  -0.24928264]
 [ 0.19986795 -1.77699256 -1.39712381  0.10153597]
 [-0.1307098  -0.6939618   0.45365265  0.33374387]
 [ 0.28339007 -1.46200562  0.26699179 -1.38850904]
 [-0.61209089  0.61759257  2.35266709 -0.67915756]
 [-0.04189507 -0.4209936   0.25579578 -0.45282298]
 [-0.4843902  -2.0866611  -0.74460214  0.72786808]
 [-0.81797385 -0.50362325 -0.05660002  2.68420768]
 [-0.38786694  0.85173392  0.29009327 -0.87774098]
 [-1.14863825 -1.63646805  0.31737217  0.39797053]]
TopKV2(values=array([[ 1.7258538 ,  1.22763908],
       [ 1.75641167, -0.75762498],
       [ 0.28343555, -0.46270299],
       [ 1.16568851,  0.70551711],
       [ 0.95638633,  0.65080678],
       [ 0.98206872,  0.08441091],
       [ 0.7031458 , -0.2947146 ],
       [ 1.97574341,  1.68389535],
       [ 0.71642655,  0.17643112],
       [ 0.59653622,  0.06258423]], dtype=float32), indices=array([[1, 3],
       [0, 3],
       [3, 0],
       [1, 3],
       [0, 3],
       [0, 2],
       [3,

In [29]:
a = [[1,2,3],[4,5,6]]
print(a)

[[1, 2, 3], [4, 5, 6]]
